https://www.kobis.or.kr/kobis/business/stat/them/findAreaShareList.do

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

# 엑셀파일 다운하는 크롤링

In [2]:
driver = webdriver.Chrome()
url = 'https://www.kobis.or.kr/kobis/business/stat/them/findAreaShareList.do'
driver.get(url)
time.sleep(1)

# 반복할 리스트 만들기
year_list = range(2013,2024)
start_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
end_month = [31,29,31,30,31,30,31,31,30,31,30,31]

for year in year_list:
    for i in range(0,12):
        
        start_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
        
        if int(str(year)[-2:])%4 == 0:
            end_month = [31,29,31,30,31,30,31,31,30,31,30,31]
        else:
            end_month = [31,28,31,30,31,30,31,31,30,31,30,31]
        
        # 변수 초기화
        start_day = 0
        end_day = 0
        
        # 리스트 조합
        start_day = str(year) + "-" + start_month[i] + "-" + '01'
        end_day = str(year) + "-" + start_month[i] + "-" + str(end_month[i])
        
        if start_day =="2023-05-01":
            break
        
        # 조회기간 지우기
        driver.find_element(By.NAME,'sSearchFrom').clear()
        time.sleep(0.5)
        driver.find_element(By.NAME,'sSearchTo').clear()
        time.sleep(0.5)

        # 조회기간 설정
        driver.find_element(By.NAME,'sSearchFrom').send_keys(start_day)
        time.sleep(0.5)
        driver.find_element(By.NAME,'sSearchTo').send_keys(end_day)
        time.sleep(0.5)


        # 조회 버튼 클릭
        driver.find_element(By.CSS_SELECTOR,'div.wrap_btn .btn_blue').click()
        time.sleep(2)

        # 엑셀 파일 다운로드
        driver.find_element(By.CSS_SELECTOR,'div.tar .btn_small').click()
        time.sleep(2)

        # 자바스크립트 확인
        Alert(driver).accept()
        time.sleep(3)

# 파일 합치기

In [27]:
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [81]:
df_list=[]
for file in glob(r'C:\workspace\City_Rock\XLSX\*.xlsx'):
    base = pd.read_excel(file, engine='openpyxl')
    연도 = base.iloc[2,0].split(':')[1].strip().split("-")[0]
    월 = base.iloc[2,0].split(':')[1].strip().split("-")[1]
    
    df = pd.read_excel(file, header=5, engine='openpyxl')
    df['연도']=연도
    df['월']=월
    
    df_list.append(df)

In [82]:
data = pd.concat(df_list)
print(data.shape)
data

(2232, 15)


,Unnamed: 0,상영편수,매출액,관객수,점유율,상영편수.1,매출액.1,관객수.1,점유율.1,상영편수.2,매출액.2,관객수.2,점유율.2,연도,월
0,서울시,61,35808207824,4760357,0.263,123,8847987791,1155037,0.310,184,44656195615,5915394,0.271,2013,02
1,경기도,20,29619862500,4064394,0.225,62,5965079000,830356,0.223,82,35584941500,4894750,0.224,2013,02
2,부산시,21,10568223500,1492051,0.083,55,2148819000,303295,0.081,76,12717042500,1795346,0.082,2013,02
3,대구시,16,7693242500,1107659,0.061,53,1513209500,216774,0.058,69,9206452000,1324433,0.061,2013,02
4,경상남도,10,6847082000,964550,0.053,41,1231742000,176583,0.047,51,8078824000,1141133,0.052,2013,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,전라남도,10,1717548500,252007,0.021,21,772457500,114005,0.014,31,2490006000,366012,0.018,2013,01
14,울산시,9,1638547500,233096,0.019,19,931495500,132013,0.016,28,2570043000,365109,0.018,2013,01
15,제주도,10,812651000,115984,0.010,17,447207000,61629,0.007,27,1259858000,177613,0.009,2013,01
16,세종시,8,101806500,16136,0.001,14,46626500,7297,0.001,22,148433000,23433,0.001,2013,01


In [91]:
data.columns=["지역","국내상영편수","국내매출액","국내관객수","국내점유율",
              "해외상영편수","해외매출액","해외관객수","해외점유율",
              "전체상영편수","전체매출액","전체관객수","전체점유율","연도","월"]
data

,지역,국내상영편수,국내매출액,국내관객수,국내점유율,해외상영편수,해외매출액,해외관객수,해외점유율,전체상영편수,전체매출액,전체관객수,전체점유율,연도,월
0,서울시,61,35808207824,4760357,0.263,123,8847987791,1155037,0.310,184,44656195615,5915394,0.271,2013,02
1,경기도,20,29619862500,4064394,0.225,62,5965079000,830356,0.223,82,35584941500,4894750,0.224,2013,02
2,부산시,21,10568223500,1492051,0.083,55,2148819000,303295,0.081,76,12717042500,1795346,0.082,2013,02
3,대구시,16,7693242500,1107659,0.061,53,1513209500,216774,0.058,69,9206452000,1324433,0.061,2013,02
4,경상남도,10,6847082000,964550,0.053,41,1231742000,176583,0.047,51,8078824000,1141133,0.052,2013,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,전라남도,10,1717548500,252007,0.021,21,772457500,114005,0.014,31,2490006000,366012,0.018,2013,01
14,울산시,9,1638547500,233096,0.019,19,931495500,132013,0.016,28,2570043000,365109,0.018,2013,01
15,제주도,10,812651000,115984,0.010,17,447207000,61629,0.007,27,1259858000,177613,0.009,2013,01
16,세종시,8,101806500,16136,0.001,14,46626500,7297,0.001,22,148433000,23433,0.001,2013,01


In [95]:
data1 = data[data['지역']!='합계']
data1

,지역,국내상영편수,국내매출액,국내관객수,국내점유율,해외상영편수,해외매출액,해외관객수,해외점유율,전체상영편수,전체매출액,전체관객수,전체점유율,연도,월
0,서울시,61,35808207824,4760357,0.263,123,8847987791,1155037,0.310,184,44656195615,5915394,0.271,2013,02
1,경기도,20,29619862500,4064394,0.225,62,5965079000,830356,0.223,82,35584941500,4894750,0.224,2013,02
2,부산시,21,10568223500,1492051,0.083,55,2148819000,303295,0.081,76,12717042500,1795346,0.082,2013,02
3,대구시,16,7693242500,1107659,0.061,53,1513209500,216774,0.058,69,9206452000,1324433,0.061,2013,02
4,경상남도,10,6847082000,964550,0.053,41,1231742000,176583,0.047,51,8078824000,1141133,0.052,2013,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,강원도,13,2084708017,287269,0.024,32,1199166000,162307,0.019,45,3283874017,449576,0.022,2013,01
13,전라남도,10,1717548500,252007,0.021,21,772457500,114005,0.014,31,2490006000,366012,0.018,2013,01
14,울산시,9,1638547500,233096,0.019,19,931495500,132013,0.016,28,2570043000,365109,0.018,2013,01
15,제주도,10,812651000,115984,0.010,17,447207000,61629,0.007,27,1259858000,177613,0.009,2013,01


In [102]:
data2 = data1[["지역","연도","월", "국내상영편수","국내매출액","국내관객수","국내점유율",
              "해외상영편수","해외매출액","해외관객수","해외점유율",
              "전체상영편수","전체매출액","전체관객수","전체점유율"]]
data2

,지역,연도,월,국내상영편수,국내매출액,국내관객수,국내점유율,해외상영편수,해외매출액,해외관객수,해외점유율,전체상영편수,전체매출액,전체관객수,전체점유율
0,서울시,2013,02,61,35808207824,4760357,0.263,123,8847987791,1155037,0.310,184,44656195615,5915394,0.271
1,경기도,2013,02,20,29619862500,4064394,0.225,62,5965079000,830356,0.223,82,35584941500,4894750,0.224
2,부산시,2013,02,21,10568223500,1492051,0.083,55,2148819000,303295,0.081,76,12717042500,1795346,0.082
3,대구시,2013,02,16,7693242500,1107659,0.061,53,1513209500,216774,0.058,69,9206452000,1324433,0.061
4,경상남도,2013,02,10,6847082000,964550,0.053,41,1231742000,176583,0.047,51,8078824000,1141133,0.052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,강원도,2013,01,13,2084708017,287269,0.024,32,1199166000,162307,0.019,45,3283874017,449576,0.022
13,전라남도,2013,01,10,1717548500,252007,0.021,21,772457500,114005,0.014,31,2490006000,366012,0.018
14,울산시,2013,01,9,1638547500,233096,0.019,19,931495500,132013,0.016,28,2570043000,365109,0.018
15,제주도,2013,01,10,812651000,115984,0.010,17,447207000,61629,0.007,27,1259858000,177613,0.009


In [103]:
data2.to_excel("지역별_점유율(2013.01~2023.04).xlsx",index=False)